In [1]:
import torch

version = torch.__version__  #查看PyTorch版本
print("torch_gpu version:", version)

torch_gpu version: 1.13.1+cpu


# GRU+Attention

In [2]:
# 1. 导入所需库
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
 
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)
 
    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(1)
        repeated_hidden = hidden.unsqueeze(1).repeat(1, max_len, 1)
        energy = torch.tanh(self.attn(torch.cat((repeated_hidden, encoder_outputs), dim=2)))
        attention_scores = self.v(energy).squeeze(2)
        attention_weights = nn.functional.softmax(attention_scores, dim=1)
        context_vector = (encoder_outputs * attention_weights.unsqueeze(2)).sum(dim=1)
        return context_vector, attention_weights

In [5]:
# 2. GRU模型构建+注意力机制
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout=0.5):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
 
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, hidden = self.gru(x, h0)
        out, attention_weights = self.attention(hidden[-1], out)
        out = self.dropout(out)
        out = self.fc(out)
        return out

In [4]:
# 3. 准备数据集
class SampleDataset(Dataset):
    def __init__(self):
        self.sequences = []
        self.labels = []
        for _ in range(1000):
            seq = torch.randn(10, 5)
            label = torch.zeros(2)
            if seq.sum() > 0:
                label[0] = 1
            else:
                label[1] = 1
            self.sequences.append(seq)
            self.labels.append(label)
 
    def __len__(self):
        return len(self.sequences)
 
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]
 
 
train_set_split = int(0.8 * len(SampleDataset()))
train_set, test_set = torch.utils.data.random_split(SampleDataset(),
                                                    [train_set_split, len(SampleDataset()) - train_set_split])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [6]:
# 4. 定义训练过程
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
 
    for batch_idx, (inputs, labels) in enumerate(loader):
        inputs, labels = inputs.to(device), labels.to(device)
 
        optimizer.zero_grad()
 
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        _, true_labels = torch.max(labels, 1)
        total += true_labels.size(0)
        correct += (predicted == true_labels).sum().item()
 
    print("Train Loss: {:.4f}, Acc: {:.2f}%".format(running_loss / (batch_idx + 1), 100 * correct / total))
 
 
# 5. 定义评估过程
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
 
    with torch.no_grad():
        for batch_idx, (inputs, labels) in enumerate(loader):
            inputs, labels = inputs.to(device), labels.to(device)
 
            outputs = model(inputs)
            loss = criterion(outputs, labels)
 
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            _, true_labels = torch.max(labels, 1)
            total += true_labels.size(0)
            correct += (predicted == true_labels).sum().item()
 
    print("Test Loss: {:.4f}, Acc: {:.2f}%".format(running_loss / (batch_idx + 1), 100 * correct / total))
 
 
# 6. 训练模型并评估
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GRUModel(input_size=5, hidden_size=10, output_size=2, num_layers=1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
 
num_epochs = 100
for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch + 1, num_epochs))
    train(model, train_loader, criterion, optimizer, device)
    evaluate(model, test_loader, criterion, device)

C:\Users\Public\TongYuan\.julia\miniforge3\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 1/100
Train Loss: 0.6976, Acc: 51.38%
Test Loss: 0.6814, Acc: 57.00%
Epoch 2/100
Train Loss: 0.6859, Acc: 54.88%
Test Loss: 0.6714, Acc: 59.00%
Epoch 3/100
Train Loss: 0.6728, Acc: 60.38%
Test Loss: 0.6588, Acc: 69.50%
Epoch 4/100
Train Loss: 0.6563, Acc: 66.00%
Test Loss: 0.6403, Acc: 76.50%
Epoch 5/100
Train Loss: 0.6324, Acc: 75.25%
Test Loss: 0.6099, Acc: 87.00%
Epoch 6/100
Train Loss: 0.5907, Acc: 81.62%
Test Loss: 0.5588, Acc: 87.00%
Epoch 7/100
Train Loss: 0.5409, Acc: 84.62%
Test Loss: 0.4783, Acc: 87.50%
Epoch 8/100
Train Loss: 0.4677, Acc: 85.00%
Test Loss: 0.3977, Acc: 89.00%
Epoch 9/100
Train Loss: 0.4176, Acc: 85.00%
Test Loss: 0.3464, Acc: 90.50%
Epoch 10/100
Train Loss: 0.3750, Acc: 87.62%
Test Loss: 0.3066, Acc: 92.50%
Epoch 11/100
Train Loss: 0.3330, Acc: 91.00%
Test Loss: 0.2672, Acc: 93.50%
Epoch 12/100
Train Loss: 0.2948, Acc: 91.00%
Test Loss: 0.2471, Acc: 93.50%
Epoch 13/100
Train Loss: 0.2746, Acc: 91.50%
Test Loss: 0.2248, Acc: 93.50%
Epoch 14/100
Train Lo